<a href="https://colab.research.google.com/github/Z3376/BT-estimate/blob/master/BT_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('./gdrive')

# Terminal

In [0]:
%%bash
pip install livelossplot
pip install keras==2.2.4
# pip install nose
# git clone 'https://github.com/tscohen/GrouPy.git'
# mv ./GrouPy/* .
# python setup.py install

# Import

In [0]:
import numpy as np
import pandas as pd
import os
import subprocess
import keras
import tensorflow as tf
from livelossplot import PlotLossesKeras
import matplotlib.pyplot as plt
from keras.preprocessing import image
from sklearn.metrics import confusion_matrix
from tqdm import tqdm
import seaborn as sns
from scipy import stats
import random
# from groupy.gconv.tensorflow_gconv.splitgconv2d import gconv2d, gconv2d_util

#Data

In [0]:
def bt(im_name):
    return float(im_name.split('_')[-1])

In [0]:
def rad(im_name):
    return float(im_name.split('_')[-3])

In [0]:
def bt_categorical(im_name):
    return str(int(float(im_name.split('_')[-1])*10))

In [0]:
if(not(os.path.isdir('./img'))):
    !unzip './gdrive/My Drive/img_highres.zip'

In [0]:
img_folder = '/content/img'

In [0]:
p = subprocess.Popen(['ls',img_folder],stdout=subprocess.PIPE)
ls = str(p.communicate()[0],'utf-8').split('.jpg\n')[:-1]

In [0]:
print(len(ls))

In [0]:
test_ls = ls[-5000:]
ls = ls[:-5000]

In [0]:
i = 0
while(i<len(ls)):
    try:
        if(ls[i][-1]==')'):
            ls.pop(i)
            i-=1
        elif(os.path.getsize(img_folder+'/'+ls[i]+'.jpg')==0):
            ls.pop(i)
            i-=1 
    except:
        print(i)
    i+=1

In [0]:
ln = len(ls)
print(len(ls))

In [0]:
label = [[ls[i]+'.jpg',bt(ls[i])] for i in range(ln)]

In [0]:
label_df = pd.DataFrame(label)
label_df.columns = ['id','bt']

In [0]:
label_df = label_df[label_df.bt!=1]
label_df = label_df[label_df.bt!=0]
ln = len(label_df)

In [0]:
train_label_df = label_df[:-5000]
val_label_df = label_df[-5000:]

# Distribution

In [0]:
dist = [0 for i in range(11)]
for i in range(len(train_label_df)):
#     ind = 9 if int(label_df.bt.iloc[i]*10)==10 else int(label_df.bt.iloc[i]*10)
    ind = int(train_label_df.bt[i]*10)
    dist[ind]+=1

In [0]:
plt.plot([i/10. for i in range(11)],dist)
plt.show()

# Generator

In [0]:
batch_size = 128
im_size = 128

In [0]:
train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale = 1./255, 
                                                             horizontal_flip = True,
                                                             vertical_flip = True, 
                                                             fill_mode = "constant",
                                                             cval = 0,
                                                             rotation_range=360) 
train_generator = train_datagen.flow_from_dataframe(dataframe = train_label_df, 
                                                    directory = img_folder, 
                                                    x_col = "id",
                                                    y_col = "bt",
                                                    has_ext = True, 
                                                    class_mode = "other",
                                                    target_size = (im_size,im_size), 
                                                    batch_size = batch_size)

In [0]:
val_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_dataframe(dataframe=val_label_df,
                                                  directory=img_folder,
                                                  x_col="id", y_col="bt",
                                                  has_ext=True,
                                                  class_mode="other",
                                                  target_size=(im_size,im_size),
                                                  batch_size=batch_size)

In [0]:
# def mask_img(img_arr,shape,mask,mask_p,return_image=True):
#     h,w,_ = shape
#     for j in range((h//2)-mask_p,(h//2)+mask_p):
#         mask[j][(w//2)-mask_p:(w//2)+mask_p] = 1
#     if(return_image):
#         img = image.array_to_img(img_arr*mask)
#     else:
#         img = img_arr*mask
#     return img

In [0]:
# def masked_generator(generator=train_generator,mask1_p=16,mask2_p=32):
#     while(True):
#         X,Y = generator.next()
#         shape = X[0].shape
#         mask = np.zeros_like(X[0])
#         func1 = lambda x:mask_img(x,shape,mask,mask1_p,return_image=False)
#         func2 = lambda x:mask_img(x,shape,mask,mask2_p,return_image=False)
#         X1 = func1(X)
#         X2 = func2(X)
#         yield ([X,X2,X1],Y)

In [0]:
def single_band_generator(generator=train_generator,band='r'):
    dic = {'i':0,'r':1,'g':2}
    while(True):
        X,Y = generator.next()
        shape = X[0].shape
        X = X[:,:,:,dic[band]]
        X = np.expand_dims(X,axis=-1)
        X = np.tile(X,[1,1,1,3])
        yield (X,Y)

In [0]:
X,Y = next(train_generator)
t_ind = random.sample(range(batch_size),8)
for i in range(len(t_ind)):
    fig = plt.figure()
    plt.imshow(X[t_ind[i]])
    # fig.savefig('single_band_'+str(i)+'.jpg')
    plt.show()
    print(Y[t_ind[i]])

#Model

In [0]:
def get_model(im_size=128):
    inpt = keras.layers.Input((im_size,im_size,3))
    base_model = keras.applications.Xception(input_tensor=inpt,weights='imagenet',include_top=False,pooling='avg')
    x = base_model.output

    out = keras.layers.Dense(1,activation='sigmoid')(x)
    model3 = keras.models.Model(inpt,out)

    return model3

In [0]:
# def get_mask_tensor(shape,mask,mask_p):
#     h,w,_ = shape
#     for j in range((h//2)-mask_p,(h//2)+mask_p):
#         mask[j][(w//2)-mask_p:(w//2)+mask_p] = 1
#     mask = tf.convert_to_tensor(mask)
#     return mask

In [0]:
# class gconv2d_start(keras.layers.Layer):
#     def __init__(self,output_channels,f,**kwargs):
#         self.n_c = output_channels
#         self.f = f
#         super(gconv2d_start,self).__init__(**kwargs)
#     def build(self,input_shape):
#         self.gconv_indices, self.gconv_shape_info, self.w_shape = gconv2d_util(h_input='Z2',
#                                                                         h_output='D4',
#                                                                         in_channels=input_shape[-1],
#                                                                         out_channels=self.n_c,
#                                                                         ksize=self.f)
#         self.kernel = self.add_weight(name='kernel',
#                                       shape=self.w_shape,
#                                       initializer='he_normal',
#                                       trainable=True)
#         super(gconv2d_start, self).build(input_shape)
#         print(input_shape[0],self.n_c,*input_shape[1:3])
#     def call(self, x):
#         print('x:',x.shape)
#         y = gconv2d(x,self.kernel,strides=[1,1,1,1],padding='SAME',gconv_indices=self.gconv_indices, gconv_shape_info=self.gconv_shape_info)
#         return y
#     def compute_output_shape(self, input_shape):
#         return (input_shape[0],self.n_c,*input_shape[1:3])

In [0]:
# class gconv2d_mid(keras.layers.Layer):
#     def __init__(self,output_channels,f,**kwargs):
#         self.n_c = output_channels
#         self.f = f
#         super(gconv2d_mid,self).__init__(**kwargs)
#     def build(self,input_shape):
#         print(input_shape)
#         self.gconv_indices, self.gconv_shape_info, self.w_shape = gconv2d_util(h_input='D4',
#                                                                         h_output='D4',
#                                                                         in_channels=input_shape[1],
#                                                                         out_channels=self.n_c,
#                                                                         ksize=self.f)
#         # print(self.gconv_shape_info)
#         self.kernel = self.add_weight(name='kernel',
#                                       shape=self.w_shape,
#                                       initializer='he_normal',
#                                       trainable=True)
#         super(gconv2d_mid, self).build(input_shape)
#     def call(self, x):
#         y = gconv2d(x,self.kernel,strides=[1,1,1,1],padding='SAME',gconv_indices=self.gconv_indices, gconv_shape_info=self.gconv_shape_info)
#         return y
#     def compute_output_shape(self, input_shape):
#         return (input_shape[0],self.n_c,*input_shape[2:])

In [0]:
# gconv_indices, gconv_shape_info, w_shape = gconv2d_util(h_input='Z2', 
#                                                             h_output='C4', 
#                                                             in_channels=n_C[layer], 
#                                                             out_channels=n_C[layer+1], 
#                                                             ksize=f[layer])
# W.append(tf.get_variable('W'+str(layer),w_shape,initializer=tf.contrib.layers.xavier_initializer()))
# Z1 = gconv2d(X,W[layer],strides=[1,1,1,1],padding='SAME',gconv_indices=gconv_indices, gconv_shape_info=gconv_shape_info)
# A1 = tf.nn.relu(Z1)

In [0]:
# keras.backend.clear_session()

In [0]:
# base_model = keras.applications.Xception(weights='imagenet',include_top=False,pooling='max')
# inpt = keras.layers.Input((im_size,im_size,3))
# r = keras.layers.Conv2D(1,(1,1),activation='relu')(inpt)
# r = keras.layers.Flatten()(r)
# r = keras.layers.Dense(1,activation='sigmoid')(r)
# att = keras.layers.Lambda(lambda ls:mask_layer(ls))([inpt,r])
# x = base_model(att)

# out = keras.layers.Dense(1,activation='sigmoid')(x)
# model3 = keras.models.Model([inpt,inpt1,inpt2],out)

In [0]:
# inpt = keras.layers.Input((im_size,im_size,3))
# x = gconv2d_start(output_channels=32,f=5)(inpt)
# # x = keras.layers.MaxPooling2D((2,2))(x)
# x = gconv2d_mid(64,3)(x)
# # x = keras.layers.MaxPooling2D((2,2))(x)
# x = gconv2d_mid(128,3)(x)
# # x = keras.layers.MaxPooling2D((2,2))(x)
# fc = keras.layers.Flatten()(x)
# out = keras.layers.Dense(1,activation='sigmoid')(fc)
# model = keras.models.Model(inpt,out)

In [0]:
# model.summary()

In [0]:
# inpt = keras.layers.Input((im_size,im_size,3))
# conv = keras.layers.Conv2D(128,(5,5),activation='relu',padding='same')(inpt)
# conv = keras.layers.Conv2D(128,(5,5),activation='relu',padding='same')(conv)
# mp = keras.layers.MaxPooling2D((2,2))(conv)
# conv = keras.layers.Conv2D(256,(3,3),activation='relu',padding='same')(mp)
# conv = keras.layers.Conv2D(256,(3,3),activation='relu',padding='same')(conv)
# mp = keras.layers.MaxPooling2D((2,2))(conv)
# conv = keras.layers.Conv2D(512,(3,3),activation='relu',padding='same')(mp)
# conv = keras.layers.Conv2D(512,(3,3),activation='relu',padding='same')(conv)
# mp = keras.layers.MaxPooling2D((2,2))(conv)
# fc = keras.layers.Flatten()(mp)

# att = keras.layers.Dense(1,activation='tanh',name='att1')(mp)
# att = keras.layers.Flatten(name='att2')(att)
# att = keras.layers.Dense(16*16,activation='softmax',name='att3')(att)

# att_out = keras.layers.Reshape((16,16))(att)
# # att_out = keras.layers.Lambda(lambda x: tf.tile(x,[1,8,8]))(att_out)
# attention = keras.models.Model(inpt,att_out)

# att = keras.layers.Lambda(lambda x: tf.tile(x,[1,512]),name='att4')(att)
# att_fc = keras.layers.Multiply(name='att5')([fc,att])

# out = keras.layers.Dense(1,activation='sigmoid')(att_fc)
# model = keras.models.Model(inpt,out)

# # attention = keras.models.Model(inpt,att_fc)

In [0]:
# att_model.summary()

In [0]:
# attention.summary()

In [0]:
# model3.summary()

In [0]:
# keras.utils.plot_model(model3,'BT_CNN.png')

# Training

## Losses

In [0]:
def exp_MAE(y_true,y_pred):
    return(keras.backend.exp(1+keras.backend.abs(y_true-y_pred))-1)

In [0]:
def RMSE(y_true,y_pred):
    return(keras.backend.sqrt(keras.backend.mean(keras.backend.square(y_true-y_pred))))

In [0]:
def RMAE(y_true,y_pred):
    return(keras.backend.sqrt(keras.backend.mean(keras.backend.abs(y_true-y_pred))))

In [0]:
def frac_err(y_true,y_pred):
    if(y_true!=0):
        return(keras.backend.abs((y_true-y_pred)/(y_true)))
    else:
        return(keras.backend.abs((y_true-y_pred)))

In [0]:
def frac_err1(y_true,y_pred):
    return(keras.backend.sqrt((keras.backend.mean(keras.backend.abs((y_true-y_pred)/(y_true+0.5))))))

In [0]:
def frac_err2(y_true,y_pred):
    return(keras.backend.sqrt((keras.backend.mean(keras.backend.abs((y_true-y_pred)/(y_true+0.1))))))

In [0]:
def frac_err_bi(y_true,y_pred):
    return(keras.backend.mean(keras.backend.abs(y_true-y_pred)/(y_true*(1-y_true))))

In [0]:
def huber_loss(y_true, y_pred):

    clip_delta = 1.0

    error = y_true - y_pred
    cond  = tf.keras.backend.abs(error) < clip_delta

    squared_loss = 0.5 * tf.keras.backend.square(error)
    linear_loss  = clip_delta * (tf.keras.backend.abs(error) - 0.5 * clip_delta)

    return tf.where(cond, squared_loss, linear_loss)

## Optimizer

In [0]:
# adam = keras.optimizers.Adam(lr=3e-5)
adam = keras.optimizers.Adam(lr=5e-4)
# adam = keras.optimizers.Adam(lr=1e-3)

## Train

In [0]:
model = get_model()

In [0]:
# model.summary()

In [0]:
# model.load_weights('./gdrive/My Drive/accnt.3/bt_cnn_cp.h5')

In [0]:
model.compile(optimizer=adam,loss='MAE',metrics=['MSE'])

csv = keras.callbacks.CSVLogger('./gdrive/My Drive/accnt.3/bt_cnn.csv')
es = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,restore_best_weights=True,verbose=1)
cp = keras.callbacks.ModelCheckpoint('./gdrive/My Drive/accnt.3/bt_cnn_cp.h5',monitor='val_loss',save_best_only=True)

history = model.fit_generator(train_generator,
                               steps_per_epoch=(len(train_label_df)//batch_size),
                               epochs=40,
                               validation_data=val_generator,
                               validation_steps=5000//batch_size,
                               callbacks=[es,cp,csv,PlotLossesKeras()])

In [0]:
model.save_weights('bt_cnn.h5')

In [0]:
log = pd.read_csv('./gdrive/My Drive/accnt.3/bt_cnn.csv')
plt.figure()
plt.plot(log['loss'])
plt.plot(log['val_loss'])
plt.grid(alpha=0.2)
plt.show()